# Type-constraint POS tagging in Keras

In this exercise you will learn how to implement a sequence prediction model in Keras.

Afterwards you will extend the tagger so to use type-constraint decoding.

    * todo: check for data, add functional, add exercise dropout and type-constr

## A simple POS tagger based on Keras

[add more intro]

We have prepared a basic skeleton for you. In order to build a tagger you need instantiate your own version `SequenceTagger`. We do this by creating a subclass `BasicSequenceTagger` from the abstract class.

In [2]:
from lib.tagger import SequenceTagger

Using Theano backend.


This is a basic abstract class that implements already some functionality, like input/output handling (`read_data`, and provides basic `fit` and `predict` functions). In oder to create your tagger you need to specify a **model** by implementing your own `build_graph` function. 

In [19]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Activation
from keras.layers.recurrent import SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding

class BasicSequenceTagger(SequenceTagger):
    
    def __init__(self):
        self.in_dim=64
        self.h_dim=100

    def build_graph(self):
        ### version1: traditional 'keras'-style code
        self.model = Sequential()
        self.model.add(Embedding(self.get_num_features(), self.in_dim,
                                 input_length=self.max_sentence_len, mask_zero=True))
        self.model.add(SimpleRNN(self.h_dim, return_sequences=True))
        self.model.add(TimeDistributed(Dense(self.get_num_tags())))
        self.model.add(Activation('softmax'))

        self.model.compile(loss='categorical_crossentropy', optimizer='adam',
                      metrics=['accuracy'])

In [16]:
tagger = BasicSequenceTagger()
tagger.read_data("data/wsj-5000.train", "data/wsj.dev")

data/wsj-5000.train


5000 sentences 118719 tokens
15249 features
max_sentence_len: 91


In [17]:
print("Build model")
tagger.build_graph()
print("train..")
batch_size=50
epochs=5
tagger.fit(batch_size, epochs)

Build model
train..
Epoch 1/5
5000/5000 [==============================] - 7s - loss: 1.7863 - acc: 0.1296     
Epoch 2/5
5000/5000 [==============================] - 7s - loss: 0.5513 - acc: 0.2255     
Epoch 3/5
5000/5000 [==============================] - 7s - loss: 0.1874 - acc: 0.2499     
Epoch 4/5
5000/5000 [==============================] - 7s - loss: 0.0985 - acc: 0.2547     
Epoch 5/5
5000/5000 [==============================] - 7s - loss: 0.0677 - acc: 0.2560     


In [18]:
# lets get predictions
tagger.predict(tagger.test_X) 

correct 124895.0 total: 131768.0 accuracy: 0.9478401432821323


// add functional version 

See here for a [tiny intro into Keras](https://github.com/bplank/ltp-notebooks/blob/master/mini-intro-Keras.ipynb)

In [2]:
# add dropout to Keras tagger

In [ ]:
# add wiktionary constraints to tagger (Keras/rungsted?)